In [2]:
import pandas as pd
import warnings as w

w.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [3]:
import base64
from requests import post
import json

client_id = 'YOUR_CLIENT_ID'
client_secret = 'YOUR_CLIENT_SECRET'

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}


token = get_token()

In [4]:
import spotipy

sp = spotipy.Spotify(auth=token)

artist_name = "Tyler, the Creator"
results = sp.search(q='artist:' + artist_name, type='artist')
artist_id = results['artists']['items'][0]['id']

artist_info = sp.artist(artist_id)
genres = artist_info['genres']
artist_image_url = artist_info['images'][0]['url']

albums = sp.artist_albums(artist_id, album_type='album')['items']

track_data = []

for album in albums:
    album_id = album['id']
    album_name = album['name']
    release_date = album['release_date']
    album_artwork_url = album['images'][0]['url']
  
    tracks = sp.album_tracks(album_id)['items']
    
    for track in tracks:
        track_info = sp.track(track['id'])
        audio_features = sp.audio_features(track['id'])[0]
        data = {
            'artist_name': artist_name,
            'artist_image_url': artist_image_url,
            'track_name': track['name'],
            'track_id': track['id'],
            'album_name': album_name,
            'album_id': album_id,
            'album_artwork_url': album_artwork_url,
            'genres': genres,
            'release_date': release_date,
            'popularity': track_info['popularity'],
            'explicit': track_info['explicit'],
            'danceability': audio_features['danceability'],
            'energy': audio_features['energy'],
            'key': audio_features['key'],
            'loudness': audio_features['loudness'],
            'mode': audio_features['mode'],
            'speechiness': audio_features['speechiness'],
            'acousticness': audio_features['acousticness'],
            'instrumentalness': audio_features['instrumentalness'],
            'liveness': audio_features['liveness'],
            'valence': audio_features['valence'],
            'tempo': audio_features['tempo'],
            'time_signature': audio_features['time_signature'],
            'featured_artists': [artist['name'] for artist in track_info['artists']],
            'type': track_info['type'],
            'duration': track_info['duration_ms'],
            'year': release_date[:4],
            'month': release_date[5:7],
            'day_of_the_week': pd.to_datetime(release_date).day_name(),
        }
        track_data.append(data)

data = pd.DataFrame(track_data)

In [7]:
#data.to_csv("TylertheCreator_forAnalysis.csv", index=False)